## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = list(range(1,1001))
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [9]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [10]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [11]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [12]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([8.20528791e-01, 6.36043803e-01, 2.81784381e+00, 3.29248389e+00,
       4.02860684e+00, 3.46202508e-01, 4.24842623e+00, 4.45602464e+00,
       6.45596563e+00, 9.85775694e+00, 7.10877417e+00, 9.71555200e+00,
       1.26756919e+01, 1.25717789e+01, 1.13857266e+01, 1.35030186e+01,
       1.51156610e+01, 2.22063494e+00, 1.12899229e+01, 1.94922635e+01,
       4.45251123e+00, 7.57423084e+00, 1.49230782e+01, 3.91292466e+00,
       2.09844050e+01, 1.27103410e+01, 2.60510005e+01, 4.30818468e+00,
       2.77982290e+01, 1.66118756e+01, 2.97189897e+01, 9.18295186e+00,
       3.03501026e+01, 2.41980325e+01, 1.71834881e+01, 2.46397347e+01,
       7.73418106e+00, 1.28084610e+01, 7.71261925e+00, 3.51012301e+01,
       3.45534443e+01, 1.71971883e+01, 3.42980998e+01, 3.10658056e+01,
       3.63555925e+01, 1.13464229e+01, 2.35481858e+01, 3.32153564e+00,
       1.78286477e+01, 8.99258011e+00, 2.55482988e+01, 4.49665046e+00,
       6.89633579e+00, 6.97379387e+00, 4.94005321e+01, 5.01121370e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [13]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)

[0.8205287914050687,
 0.6360438030177014,
 2.8178438058589013,
 3.2924838885590333]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [14]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [15]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.7548864880926632,
 0.5851602987762853,
 2.5924163013901893,
 3.0290851774743106,
 3.7063182961006915,
 0.3185063076401547,
 3.9085521292437275,
 4.0995426704382565,
 5.939488382689959,
 9.069136382562881]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [16]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [17]:
discounted.take(10)

[0.6793978392833969,
 0.5266442688986568,
 2.3331746712511703,
 2.7261766597268795,
 3.3356864664906225,
 0.28665567687613924,
 3.517696916319355,
 3.689588403394431,
 5.345539544420963,
 8.162222744306593]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [18]:
price_items.map(sales_tax).map(lambda x: x*0.9).top(15)

[786.2776082529579,
 786.1229794833177,
 779.6790437188138,
 761.3191072179771,
 740.4581217766095,
 725.9846677222552,
 722.8657130520367,
 722.0215243287801,
 721.1517235080752,
 715.0014443330211,
 713.1039111383292,
 712.0837070167464,
 702.7356356265942,
 696.7369202241597,
 695.6407628766422]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [19]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [20]:
mapped = price_items.map(lambda x: (x, x*0.92*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.8205287914050687, 0.6793978392833969), (0.6360438030177014, 0.5266442688986568), (2.8178438058589013, 2.3331746712511703), (3.2924838885590333, 2.7261766597268795), (4.028606843587708, 3.3356864664906225), (0.34620250830451593, 0.28665567687613924), (4.248426227438834, 3.517696916319355), (4.456024641780713, 3.689588403394431), (6.455965633358651, 5.345539544420963), (9.857756937568348, 8.162222744306593)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [21]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.8205287914050687, 0.6793978392833969, 0.6360438030177014, 0.5266442688986568, 2.8178438058589013, 2.3331746712511703, 3.2924838885590333, 2.7261766597268795, 4.028606843587708, 3.3356864664906225]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [23]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

295

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [24]:
selected_items.reduce(lambda x, y: x+y)

139018.55885027975

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [25]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(17, 311.8713706348442),
 (49, 309.9035849831836),
 (38, 325.1517271412371),
 (33, 316.8140430293942),
 (6, 319.72713184317),
 (13, 362.7921254099494),
 (36, 319.8918364469401)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [26]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y: x+y)
total_spent.take(10)

[(10, 4523.305759680955),
 (50, 2191.392529195444),
 (40, 3526.73318237196),
 (20, 3640.2737184198295),
 (30, 2669.5043453697263),
 (11, 2246.035266314021),
 (31, 3375.4286310245698),
 (41, 3340.5549346636517),
 (21, 2706.068195999124),
 (1, 1993.1525441641857)]

In [28]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(19, 5547.088284053778),
 (47, 5204.431167901316),
 (46, 5035.491991315179),
 (39, 4531.101360822643),
 (10, 4523.305759680955),
 (5, 4338.335264243828),
 (15, 3990.0243596942264),
 (35, 3965.7952879320555),
 (17, 3850.05550965461),
 (8, 3816.278404809366),
 (24, 3733.02058263033),
 (20, 3640.2737184198295),
 (34, 3589.708070486161),
 (40, 3526.73318237196),
 (4, 3493.6514213834666),
 (42, 3454.4740265228565),
 (31, 3375.4286310245698),
 (36, 3347.7301447425994),
 (41, 3340.5549346636517),
 (2, 3239.138489963104),
 (28, 3214.5948314333436),
 (9, 3170.022252340089),
 (43, 2860.9427527037096),
 (21, 2706.068195999124),
 (38, 2693.8328736993158),
 (30, 2669.5043453697263),
 (32, 2650.9528471727035),
 (37, 2629.4815955675335),
 (23, 2446.4035181638715),
 (33, 2375.7861693018162),
 (16, 2361.1289925325796),
 (14, 2298.8609187099937),
 (11, 2246.035266314021),
 (50, 2191.392529195444),
 (12, 2011.9498584281362),
 (1, 1993.1525441641857),
 (13, 1929.5844195201341),
 (44, 1884.0061090179913),

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [29]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda kv:kv[1], reverse=True)

[(14, 12),
 (50, 11),
 (30, 10),
 (39, 10),
 (42, 9),
 (9, 9),
 (8, 9),
 (24, 9),
 (20, 9),
 (44, 9),
 (17, 7),
 (26, 7),
 (40, 7),
 (29, 7),
 (36, 7),
 (11, 6),
 (18, 6),
 (49, 6),
 (3, 6),
 (32, 6),
 (21, 6),
 (4, 6),
 (5, 6),
 (6, 6),
 (7, 6),
 (22, 6),
 (13, 6),
 (19, 6),
 (46, 6),
 (38, 5),
 (27, 5),
 (33, 5),
 (41, 5),
 (16, 5),
 (43, 5),
 (28, 5),
 (10, 4),
 (25, 4),
 (48, 4),
 (2, 4),
 (45, 4),
 (35, 4),
 (1, 4),
 (47, 4),
 (23, 3),
 (15, 3),
 (12, 3),
 (31, 2),
 (34, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
